In [1]:
!cp "/content/drive/MyDrive/Authorship Bias Paper/Data/finalData.csv" .

In [2]:
!pip install fuzzywuzzy

In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
def extractLastNames(arr2):
    arr3=[]
    for i in range(0,len(arr2)):
            val=arr2[i].strip().lower().split(" ")
            arr3.append(val[-1].strip())
    return arr3

In [5]:
similarity = [50, 60, 70, 80]

In [6]:
df = pd.read_csv("finalData.csv")
df.columns

Index(['Name', 'Google Scholar ID', 'Gender', 'Ethnicity', 'Affiliation',
       'Country', 'h-index', 'Citation count',
       'Co-authors’ names (Google Scholar)',
       'Co-authors’ genders (Google Scholar)',
       'Co-authors’ ethnicity (Google Scholar)', 'Co-authors’ names (OpenAI)',
       'Co-authors’ genders (OpenAI)', 'Co-authors’ ethnicity (OpenAI)'],
      dtype='object')

In [7]:
df["Match Count 50%"] = 0
df["Match Count 60%"] = 0
df["Match Count 70%"] = 0
df["Match Count 80%"] = 0

In [8]:
for limit in similarity:
  k=0
  i = 0
  arr1 = arr2 = arr3 = arr4 = []

  while k < len(df):
          try:
              arrAI = df['Co-authors’ names (OpenAI)'][k].split(', ')
              arrGS = df['Co-authors’ names (Google Scholar)'][k].split(', ')
          except:
              arrAI = arrGS = []
          arr1 = extractLastNames(arrAI)
          arr2 = extractLastNames(arrGS)
          i = 0
          matches = []
          count = 0
          maxLengthA1 = len(arr1)
          while i < maxLengthA1:
              j=0
              maxFuzz = 0
              index = 0
              foundMatch = False
              maxLengthA2 = len(arr2)
              while j < maxLengthA2:
                  currentFuzz = fuzz.ratio(arr1[i],arr2[j])
                  if currentFuzz >= limit:
                      if currentFuzz > maxFuzz:
                          maxFuzz = currentFuzz
                          index = j
                          foundMatch = True
                  j=j+1
              if foundMatch:
                  matches.append(arr2[index].lower().strip())
                  count = count + 1
                  del arr2[index]
              i=i+1
          colName = "Match Count " + str(limit) + "%"
          df.at[k, colName] = count
          k = k + 1

In [9]:
df.to_csv("matchData.csv", index = False)

In [10]:
!mv "/content/matchData.csv" "/content/drive/MyDrive/Authorship Bias Paper/Data/matchData.csv"